In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

# ----------------------------
# PARTE 1: CALCOLO PER LE VARIABILI NUMERICHE
# ----------------------------

# Percorso del file con i dati dei pazienti
data_file = r"C:\Users\Lorenzo\Desktop\scoring\demographics/summary_extracted.xlsx"
df = pd.read_excel(data_file)

# Assicuriamoci che la colonna Group sia di tipo categoria
df['Group'] = df['Group'].astype('category')

# Lista delle variabili numeriche (continue e ordinali)
numeric_vars = ['Age', 'Disease_Duration', 'LEDD', 'UPDRSIV4.1',
                'UPDRSI', 'UPDRSII', 'UPDRSIII', 'UPDRSIV', 'AIMs']

# Otteniamo i gruppi presenti
groups = df['Group'].unique()

results = []
for var in numeric_vars:
    result_row = {}
    result_row['Variable'] = var

    # --- Normalità (Test Shapiro-Wilk per ogni gruppo) ---
    normality_pvalues = {}
    decisions = []  # "Normal", "Non-normal", "Insufficient"
    for grp in groups:
        data_grp = df[df['Group'] == grp][var].dropna()
        if len(data_grp) < 3:
            normality_pvalues[grp] = "n<3"
            decisions.append("Insufficient")
        else:
            # Check for zero variance
            if data_grp.var() == 0:
                normality_pvalues[grp] = "zero variance"
                decisions.append("Non-normal")  # Zero variance means non-normal
            else:
                try:
                    stat, p = stats.shapiro(data_grp)
                    normality_pvalues[grp] = round(p, 3)
                    decisions.append("Normal" if p >= 0.05 else "Non-normal")
                except Exception as e:
                    normality_pvalues[grp] = f"Error: {str(e)}"
                    decisions.append("Error")
                    
    if all(dec == "Insufficient" for dec in decisions):
        overall_normality = "Insufficient data"
    elif any(dec == "Non-normal" for dec in decisions if dec != "Insufficient"):
        overall_normality = "Non-normal"
    else:
        overall_normality = "Normal"
    result_row['Normality'] = overall_normality
    pvals_str = "; ".join([f"{grp}: {normality_pvalues[grp]}" for grp in groups])
    result_row['Shapiro p-values'] = pvals_str

    # --- Omogeneità delle varianze (Test di Levene) ---
    samples = []
    valid_groups = []
    for grp in groups:
        data_grp = df[df['Group'] == grp][var].dropna()
        if len(data_grp) > 1:
            samples.append(data_grp.values)
            valid_groups.append(grp)
    if len(samples) < 2:
        variance_status = "Insufficient data"
        levene_p = "NA"
    else:
        try:
            stat_levene, p_levene = stats.levene(*samples)
            levene_p = round(p_levene, 3)
            variance_status = "Homogeneous" if p_levene >= 0.05 else "Heterogeneous"
        except Exception as e:
            variance_status = f"Error: {str(e)}"
            levene_p = "Error"
    result_row['Variance Homogeneity'] = variance_status
    result_row['Levene p-value'] = levene_p

    results.append(result_row)

numeric_summary_df = pd.DataFrame(results)
numeric_summary_df = numeric_summary_df[['Variable', 'Shapiro p-values', 'Normality',
                                         'Levene p-value', 'Variance Homogeneity']]

In [2]:
desired_order = ['CTL', 'DNV', 'ADV', 'DYS']
cat_results = []

for grp in desired_order:
    if grp not in df['Group'].unique():
        continue
    group_data = df[df['Group'] == grp]
    total = len(group_data)
    
    # 1) Gender: nF / nM
    if 'Gender' in df.columns:
        n_F = sum(group_data['Gender'] == 'F')
        n_M = sum(group_data['Gender'] == 'M')
        gender_summary = f"{n_F} F / {n_M} M"
        cat_results.append({
            "Group": grp,
            "Variable": "Gender",
            "Summary": gender_summary
        })
    
    # 2) DA_AGONIST_YES1NOT0: n (x%)
    if 'DA_AGONIST_YES1NOT0' in df.columns:
        n_yes = sum(group_data['DA_AGONIST_YES1NOT0'] == 1)
        pct_yes = 0 if total == 0 else round((n_yes / total) * 100, 1)
        da_summary = f"{n_yes} ({pct_yes}%)"
        cat_results.append({
            "Group": grp,
            "Variable": "DA_AGONIST_YES1NOT0",
            "Summary": da_summary
        })
    
    # 3) BDZ_YES1NOT0: n (x%)
    if 'BDZ_YES1NOT0' in df.columns:
        n_yes = sum(group_data['BDZ_YES1NOT0'] == 1)
        pct_yes = 0 if total == 0 else round((n_yes / total) * 100, 1)
        bdz_summary = f"{n_yes} ({pct_yes}%)"
        cat_results.append({
            "Group": grp,
            "Variable": "BDZ_YES1NOT0",
            "Summary": bdz_summary
        })
    
    # 4) Hoehn & Yahr: solo stadi 1, 2, 3
    if 'Hoen_Year' in df.columns:
        for stage in [1, 2, 3]:
            n_stage = sum(group_data['Hoen_Year'] == stage)
            pct_stage = 0 if total == 0 else round((n_stage / total) * 100, 1)
            hy_summary = f"{n_stage} ({pct_stage}%)"
            cat_results.append({
                "Group": grp,
                "Variable": f"Hoehn & Yahr = {stage}",
                "Summary": hy_summary
            })

categorical_summary_df = pd.DataFrame(cat_results)

In [7]:
# ----------------------------
# PARTE 3: TEST STATISTICI PER LE VARIABILI CATEGORICHE
# ----------------------------

cat_test_results = []
cat_test_vars = ['Gender', 'DA_AGONIST_YES1NOT0', 'BDZ_YES1NOT0', 'Hoen_Year']

for var in cat_test_vars:
    if var not in df.columns:
        continue
    
    # Creiamo una copia temporanea del dataframe per le operazioni sulla variabile corrente
    # In questo modo evitiamo di modificare il dataframe originale
    temp_df = df.copy()
    
    # Converto la variabile in stringa per evitare problemi di confronto
    temp_df[var] = temp_df[var].astype(str)
    
    # Creo una tabella di contingenza
    try:
        contingency = pd.crosstab(temp_df['Group'], temp_df[var])
        
        # Filtro le righe per i gruppi in desired_order se esistono
        existing_rows = [g for g in desired_order if g in contingency.index]
        if existing_rows:
            contingency = contingency.loc[existing_rows]
            
        # Non ordino le colonne per evitare errori di confronto
        # Verifico se è possibile eseguire il test Chi-square
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            # Solo per sicurezza, verifico che la tabella non abbia dimensione zero
            if contingency.size > 0:
                try:
                    chi2, p_val, dof, expected = stats.chi2_contingency(contingency)
                    
                    # Verifico se ci sono celle con valori attesi piccoli
                    expected_min = expected.min()
                    cells_under_5 = (expected < 5).sum()
                    pct_under_5 = cells_under_5 / expected.size * 100
                    
                    test_note = ""
                    if pct_under_5 > 20 or expected_min < 1:
                        test_name = "Chi-square (warning: low expected counts)"
                        test_note = f"{pct_under_5:.1f}% cells with expected count < 5"
                    else:
                        test_name = "Chi-square"
                    
                    test_result = {
                        "Variable": var,
                        "Test Used": test_name,
                        "Statistic": f"chi2={chi2:.3f}, dof={dof}",
                        "p-value": round(p_val, 3),
                        "Note": test_note
                    }
                except Exception as e:
                    test_result = {
                        "Variable": var,
                        "Test Used": "Error",
                        "Statistic": "Error",
                        "p-value": "NA",
                        "Note": f"Error in chi-square: {str(e)}"
                    }
            else:
                test_result = {
                    "Variable": var,
                    "Test Used": "Error",
                    "Statistic": "Error",
                    "p-value": "NA",
                    "Note": "Empty contingency table"
                }
        else:
            test_result = {
                "Variable": var,
                "Test Used": "Error",
                "Statistic": "Error",
                "p-value": "NA",
                "Note": "Insufficient categories in contingency table"
            }
    except Exception as e:
        test_result = {
            "Variable": var,
            "Test Used": "Error",
            "Statistic": "Error",
            "p-value": "NA",
            "Note": f"Error: {str(e)}"
        }

    cat_test_results.append(test_result)

# Crea il dataframe dei risultati del test Chi-quadrato
chi_square_df = pd.DataFrame(cat_test_results)

# Salvataggio dei risultati su file Excel
output_file = r"C:\Users\Lorenzo\Desktop\scoring/analysis_results.xlsx"
with pd.ExcelWriter(output_file) as writer:
    numeric_summary_df.to_excel(writer, sheet_name='Numerical Variables Analysis', index=False)
    categorical_summary_df.to_excel(writer, sheet_name='Categorical Variables Summary', index=False)
    chi_square_df.to_excel(writer, sheet_name='Chi-Square Tests', index=False)

print(f"Analysis results saved to: {output_file}")

Analysis results saved to: C:\Users\Lorenzo\Desktop\scoring/analysis_results.xlsx


In [6]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import warnings
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp  # per il Dunn test

# ============================
# FUNZIONI AUSILIARIE
# ============================

def pairwise_ttests_welch(data, groups, var, alpha=0.05):
    """
    Esegue confronti a coppie con t-test di Welch, applicando la correzione Bonferroni.
    Restituisce una stringa con le comparazioni significative, usando notazione scientifica per i p-value.
    (Non usata se si utilizza esclusivamente Tukey HSD per dati omogenei.)
    """
    comparisons = []
    pvals = []
    group_list = list(groups)
    for i in range(len(group_list)):
        for j in range(i+1, len(group_list)):
            grp1 = group_list[i]
            grp2 = group_list[j]
            d1 = data[data['Group'] == grp1][var].dropna()
            d2 = data[data['Group'] == grp2][var].dropna()
            if len(d1) < 2 or len(d2) < 2:
                p = np.nan
            else:
                _, p = stats.ttest_ind(d1, d2, equal_var=False)
            comparisons.append(f"{grp1} vs {grp2}")
            pvals.append(p)
    comparisons = np.array(comparisons)
    pvals = np.array(pvals, dtype=float)
    if len(pvals) > 0:
        pvals_corr = np.minimum(pvals * len(pvals), 1.0)
    else:
        pvals_corr = pvals
    significant = []
    for comp, pcorr in zip(comparisons, pvals_corr):
        if not np.isnan(pcorr) and pcorr < alpha:
            p_str = np.format_float_scientific(pcorr, precision=3)
            significant.append(f"{comp} (p={p_str})")
    return "; ".join(significant) if significant else "None"

def pairwise_dunn(data, var, groups_used, desired_order, group_col='Group', alpha=0.05):
    """
    Esegue il Dunn test con correzione Bonferroni integrata usando sp.posthoc_dunn.
    Restituisce le comparazioni significative, formattate in notazione scientifica.
    """
    sub_df = data[data[group_col].isin(groups_used)][[var, group_col]].dropna()
    if len(sub_df) < 2 or len(groups_used) < 2:
        return "Insufficient data"
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning, module="scikit_posthocs")
        try:
            dunn_res = sp.posthoc_dunn(sub_df, val_col=var, group_col=group_col, p_adjust='bonferroni')
        except Exception as e:
            return f"Error in Dunn test: {e}"
    sig = []
    groups_sorted = sorted(groups_used, key=lambda x: desired_order.index(x) if x in desired_order else len(desired_order))
    for i in range(len(groups_sorted)):
        for j in range(i+1, len(groups_sorted)):
            p_val = dunn_res.loc[groups_sorted[i], groups_sorted[j]]
            if p_val < alpha:
                p_str = np.format_float_scientific(p_val, precision=3)
                sig.append(f"{groups_sorted[i]} vs {groups_sorted[j]} (p={p_str})")
    return "; ".join(sig) if sig else "None"

def pairwise_dunn_bonferroni(data, var, groups_used, desired_order, group_col='Group', alpha=0.05):
    """
    Esegue il Dunn test senza correzione e poi applica manualmente la correzione Bonferroni.
    Restituisce le comparazioni significative, con i p-value in notazione scientifica.
    """
    sub_df = data[data[group_col].isin(groups_used)][[var, group_col]].dropna()
    if len(sub_df) < 2 or len(groups_used) < 2:
        return "Insufficient data"
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning, module="scikit_posthocs")
        try:
            dunn_res = sp.posthoc_dunn(sub_df, val_col=var, group_col=group_col, p_adjust=None)
        except Exception as e:
            return f"Error in Dunn test: {e}"
    n = len(groups_used)
    n_comparisons = n * (n - 1) / 2
    bonf_pvals = dunn_res * n_comparisons
    bonf_pvals = bonf_pvals.clip(upper=1.0)
    sig = []
    groups_sorted = sorted(groups_used, key=lambda x: desired_order.index(x) if x in desired_order else len(desired_order))
    for i in range(len(groups_sorted)):
        for j in range(i+1, len(groups_sorted)):
            p_val = bonf_pvals.loc[groups_sorted[i], groups_sorted[j]]
            if p_val < alpha:
                p_str = np.format_float_scientific(p_val, precision=3)
                sig.append(f"{groups_sorted[i]} vs {groups_sorted[j]} (p={p_str})")
    return "; ".join(sig) if sig else "None"

def welch_anova(samples, epsilon=1e-6):
    """
    Calcola Welch's ANOVA per una lista di array (samples).
    Se un gruppo ha varianza zero, viene sostituita con 'epsilon'.
    Ritorna F e p-value.
    """
    k = len(samples)
    if k < 2 or any(len(s) < 2 for s in samples):
        return np.nan, np.nan
    ns = np.array([len(s) for s in samples])
    means = np.array([np.mean(s) for s in samples])
    variances = np.array([np.var(s, ddof=1) for s in samples])
    variances = np.where(variances == 0, epsilon, variances)
    weights = ns / variances
    grand_mean = np.sum(weights * means) / np.sum(weights)
    numerator = np.sum(weights * (means - grand_mean)**2)
    F = numerator / (k - 1)
    denominator_terms = (1 - weights/np.sum(weights))**2 / (ns - 1)
    if np.sum(denominator_terms) == 0:
        return np.nan, np.nan
    df_den = (k**2 - 1) / (3 * np.sum(denominator_terms))
    if not np.isfinite(F) or not np.isfinite(df_den) or df_den <= 0:
        return np.nan, np.nan
    p_val = 1 - stats.f.cdf(F, k - 1, df_den)
    return F, p_val

def group_descriptive_stats_dict(data, groups, var, norm_status):
    """
    Calcola le statistiche descrittive per la variabile 'var' per ogni gruppo.
    Se i dati sono normali: restituisce "mean ± sd".
    Se i dati NON sono normali: restituisce "median (IQR)", dove IQR = Q3 - Q1.
    Se il gruppo è costante (std == 0) o mancano dati, restituisce "-".
    Ritorna un dizionario { gruppo: stringa }.
    """
    d = {}
    for grp in groups:
        vals = data[data['Group'] == grp][var].dropna()
        if len(vals) == 0 or np.std(vals, ddof=1) == 0:
            d[grp] = "-"
        else:
            if norm_status == "Normal":
                d[grp] = f"{np.round(np.mean(vals),2)} ± {np.round(np.std(vals, ddof=1),2)}"
            else:
                median_val = np.median(vals)
                q1 = np.percentile(vals, 25)
                q3 = np.percentile(vals, 75)
                iqr = q3 - q1
                d[grp] = f"{np.round(median_val,2)} ({np.round(iqr,2)})"
    return d

In [9]:
# ============================
# Caricamento dei dati
# ============================

data_dir = r"C:\Users\Lorenzo\Desktop\scoring\demographics"
data_file = os.path.join(data_dir, "summary_extracted.xlsx")
nv_file = os.path.join(data_dir, "normality_variance_summary.xlsx")
output_file = os.path.join(data_dir, "final_analysis.xlsx")

if not os.path.exists(data_file):
    print(f"ERRORE: File dati non trovato: {data_file}")
    exit(1)
if not os.path.exists(nv_file):
    print(f"ERRORE: File normalità/varianza non trovato: {nv_file}")
    exit(1)

try:
    df = pd.read_excel(data_file)
    df['Group'] = df['Group'].astype('category')
    nv_df = pd.read_excel(nv_file)
except Exception as e:
    print(f"ERRORE nel caricamento dei file: {e}")
    exit(1)

numeric_vars = ['Age', 'Disease_Duration', 'LEDD', 'UPDRSIV4.1', 'UPDRSIV4.2', 'UPDRSIV4.3']
numeric_vars = [var for var in numeric_vars if var in df.columns]

# ============================
# Statistiche descrittive
# ============================

descriptive_stats = {}
for var in numeric_vars:
    norm_status = nv_df.loc[nv_df['Variable'] == var, 'Normality'].values[0]
    descriptive_stats[var] = group_descriptive_stats_dict(df, df['Group'].unique(), var, norm_status)
    
# ============================
# Test Statistici
# ============================

test_results = {}
for var in numeric_vars:
    norm_status = nv_df.loc[nv_df['Variable'] == var, 'Normality'].values[0]
    if norm_status == "Normal":
        result = stats.f_oneway(*(df[df['Group'] == grp][var].dropna() for grp in df['Group'].unique()))
        test_results[var] = result
    else:
        result = stats.kruskal(*(df[df['Group'] == grp][var].dropna() for grp in df['Group'].unique()))
        test_results[var] = result

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_9316\3910445438.py:45: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  result = stats.f_oneway(*(df[df['Group'] == grp][var].dropna() for grp in df['Group'].unique()))
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_9316\3910445438.py:48: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  result = stats.kruskal(*(df[df['Group'] == grp][var].dropna() for grp in df['Group'].unique()))


In [11]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Percorsi ai file
demographics_path = r"C:\Users\Lorenzo\Desktop\scoring\demographics/summary_extracted.xlsx"
sleep_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"

# Cartella di destinazione per i grafici di correlazione
correlation_folder = r"C:\Users\Lorenzo\Desktop\scoring/correlation"
os.makedirs(correlation_folder, exist_ok=True)

# Carica i dati
df_dem = pd.read_excel(demographics_path)
df_sleep = pd.read_excel(sleep_path)

# Unione dei DataFrame: usa "ID" per i demografici e "Patient" per lo scoring
df_merged = pd.merge(df_dem, df_sleep, left_on="ID", right_on="Patient")

# Escludi i controlli (gruppo "CTL")
df_merged = df_merged[df_merged["Group_x"] != "CTL"]

# Calcolo della correlazione di Pearson utilizzando solo i dati dei pazienti (non controlli)
r, p_value = pearsonr(df_merged["Disease_Duration"], df_merged["Total Sleep Time (min)"])
print("Correlazione Pearson: r = {:.3f}, p-value = {:.3f}".format(r, p_value))

# Creazione del grafico di dispersione con linea di regressione
plt.figure(figsize=(8, 6))
sns.regplot(x="Disease_Duration", y="Total Sleep Time (min)", data=df_merged, ci=95)
plt.title("All Patients")
plt.xlabel("Disease Duration")
plt.ylabel("Total Sleep Time (min)")
plt.grid(True, linestyle="--", alpha=0.7)

# Aggiungi l'annotazione dei risultati di Pearson nell'angolo in alto a destra
plt.text(0.95, 0.95, f"r = {r:.3f}\\np = {p_value:.3f}", 
         transform=plt.gca().transAxes, 
         horizontalalignment='right', 
         verticalalignment='top', 
         fontsize=12,
         )

# Salvataggio del grafico nella cartella "correlation"
output_path = os.path.join(correlation_folder, "disease_duration_vs_tst_no_CTL.png")
plt.savefig(output_path, dpi=300, bbox_inches="tight")
plt.close()

print(f"Grafico salvato in: {output_path}")

# Lista dei gruppi da analizzare
groups = ["DNV", "ADV", "DYS"]

for group in groups:
    # Filtra il DataFrame per il gruppo corrente
    df_group = df_merged[df_merged["Group_x"] == group]
    
    # Calcola la correlazione di Pearson per il gruppo corrente
    r, p_value = pearsonr(df_group["Disease_Duration"], df_group["Total Sleep Time (min)"])
    print(f"Gruppo {group} - Correlazione Pearson: r = {r:.3f}, p-value = {p_value:.3f}")
    
    # Crea il grafico di dispersione con linea di regressione
    plt.figure(figsize=(8, 6))
    sns.regplot(x="Disease_Duration", y="Total Sleep Time (min)", data=df_group, ci=95)
    plt.title(f"{group}")
    plt.xlabel("Disease Duration")
    plt.ylabel("Total Sleep Time (min)")
    plt.grid(True, linestyle="--", alpha=0.7)
    
    # Aggiungi l'annotazione dei risultati di Pearson in alto a destra
    plt.text(0.95, 0.95, f"r = {r:.3f}\\np = {p_value:.3f}", 
             transform=plt.gca().transAxes, 
             horizontalalignment='right', 
             verticalalignment='top', 
             fontsize=12,
             bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    
    # Salva il grafico per il gruppo corrente
    output_path = os.path.join(correlation_folder, f"disease_duration_vs_tst_{group}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    
    print(f"Grafico salvato in: {output_path}")


Correlazione Pearson: r = -0.472, p-value = 0.023
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_tst_no_CTL.png
Gruppo DNV - Correlazione Pearson: r = 0.730, p-value = 0.161
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_tst_DNV.png
Gruppo ADV - Correlazione Pearson: r = -0.696, p-value = 0.055
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_tst_ADV.png
Gruppo DYS - Correlazione Pearson: r = -0.324, p-value = 0.362
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_tst_DYS.png


In [8]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Percorsi ai file
demographics_path = r"C:\Users\Lorenzo\Desktop\scoring\demographics/summary_extracted.xlsx"
sleep_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"

# Cartella di destinazione per i grafici di correlazione
correlation_folder = r"C:\Users\Lorenzo\Desktop\scoring/correlation"
os.makedirs(correlation_folder, exist_ok=True)

# Carica i dati
df_dem = pd.read_excel(demographics_path)
df_sleep = pd.read_excel(sleep_path)

# Unione dei DataFrame: "ID" per i demografici e "Patient" per lo scoring
df_merged = pd.merge(df_dem, df_sleep, left_on="ID", right_on="Patient")

# Escludi i controlli (gruppo "CTL")
df_merged = df_merged[df_merged["Group_x"] != "CTL"]

# ANALISI: Sleep Efficiency (%) vs Disease Duration

# 1. Analisi per tutti i pazienti (escludendo CTL) utilizzando Spearman
r_all, p_all = spearmanr(df_merged["Disease_Duration"], df_merged["Sleep Efficiency (%)"])
print(f"All patients - Spearman correlation: r = {r_all:.3f}, p = {p_all:.3f}")

plt.figure(figsize=(8, 6))
sns.regplot(x="Disease_Duration", y="Sleep Efficiency (%)", data=df_merged, ci=95)
plt.title("All Patients (non-CTL)")
plt.xlabel("Disease Duration")
plt.ylabel("Sleep Efficiency (%)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.text(0.95, 0.95, f"r = {r_all:.3f}\\np = {p_all:.3f}", 
         transform=plt.gca().transAxes, 
         horizontalalignment='right', 
         verticalalignment='top', 
         fontsize=12,
         bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
output_path_all = os.path.join(correlation_folder, "disease_duration_vs_sleep_efficiency_no_CTL_all.png")
plt.savefig(output_path_all, dpi=300, bbox_inches="tight")
plt.close()
print(f"Grafico salvato in: {output_path_all}")

# 2. Analisi per ciascun gruppo: DNV, ADV, DYS
groups = ["DNV", "ADV", "DYS"]

for group in groups:
    df_group = df_merged[df_merged["Group_x"] == group]
    r_group, p_group = spearmanr(df_group["Disease_Duration"], df_group["Sleep Efficiency (%)"])
    print(f"Gruppo {group} - Spearman correlation: r = {r_group:.3f}, p = {p_group:.3f}")
    
    plt.figure(figsize=(8, 6))
    sns.regplot(x="Disease_Duration", y="Sleep Efficiency (%)", data=df_group, ci=95)
    plt.title(f"Group: {group}")
    plt.xlabel("Disease Duration")
    plt.ylabel("Sleep Efficiency (%)")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.text(0.95, 0.95, f"r = {r_group:.3f}\\np = {p_group:.3f}", 
             transform=plt.gca().transAxes, 
             horizontalalignment='right', 
             verticalalignment='top', 
             fontsize=12,
             bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    output_path_group = os.path.join(correlation_folder, f"disease_duration_vs_sleep_efficiency_{group}.png")
    plt.savefig(output_path_group, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Grafico salvato in: {output_path_group}")

# Nuovo blocco di codice

# Percorsi ai file
demographics_path = r"C:\Users\Lorenzo\Desktop\scoring/demographics/summary_extracted.xlsx"
sleep_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"

# Cartella di destinazione per i grafici di correlazione
correlation_folder = r"C:\Users\Lorenzo\Desktop\scoring/correlation"
os.makedirs(correlation_folder, exist_ok=True)

# Carica i dati
df_dem = pd.read_excel(demographics_path)
df_sleep = pd.read_excel(sleep_path)

# Unione dei DataFrame: "ID" per i demografici e "Patient" per lo scoring
df_merged = pd.merge(df_dem, df_sleep, left_on="ID", right_on="Patient")

# Consideriamo solo i gruppi di interesse: DNV, ADV, DYS (utilizzando la colonna "Group_x" dei dati demografici)
groups = ["DNV", "ADV", "DYS"]
df_filtered = df_merged[df_merged["Group_x"].isin(groups)]

# Itera sui gruppi per calcolare la correlazione tra LEDD e REM% 
for group in groups:
    df_group = df_filtered[df_filtered["Group_x"] == group].dropna(subset=["LEDD", "REM%"])
    
    if len(df_group) < 3:
        print(f"Gruppo {group} ha meno di 3 soggetti validi, salto il calcolo.")
        continue
    
    r, p_value = spearmanr(df_group["LEDD"], df_group["REM%"])
    print(f"Gruppo {group} - Spearman correlation: r = {r:.3f}, p-value = {p_value:.3f}, n = {len(df_group)}")
    
    # Creazione del grafico di dispersione con linea di regressione
    plt.figure(figsize=(8, 6))
    sns.regplot(x="LEDD", y="REM%", data=df_group, ci=95)
    plt.title(f"Correlazione LEDD vs REM% in gruppo {group}")
    plt.xlabel("LEDD")
    plt.ylabel("REM (%)")
    plt.grid(True, linestyle="--", alpha=0.7)
    
    # Annotazione dei risultati di Spearman in alto a destra
    plt.text(0.95, 0.95, f"r = {r:.3f}\\np = {p_value:.3f}",
             transform=plt.gca().transAxes,
             horizontalalignment="right",
             verticalalignment="top",
             fontsize=12,
             bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))
    
    output_path = os.path.join(correlation_folder, f"LEDD_vs_REMperc_{group}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Grafico salvato in: {output_path}")


All patients - Spearman correlation: r = -0.360, p = 0.092
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_sleep_efficiency_no_CTL_all.png
Gruppo DNV - Spearman correlation: r = 0.100, p = 0.873
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_sleep_efficiency_DNV.png
Gruppo ADV - Spearman correlation: r = -0.119, p = 0.779
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_sleep_efficiency_ADV.png
Gruppo DYS - Spearman correlation: r = -0.158, p = 0.663
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\disease_duration_vs_sleep_efficiency_DYS.png
Gruppo DNV - Spearman correlation: r = 0.000, p-value = 1.000, n = 5
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\LEDD_vs_REMperc_DNV.png
Gruppo ADV - Spearman correlation: r = 0.494, p-value = 0.213, n = 8
Grafico salvato in: C:\Users\Lorenzo\Desktop\scoring/correlation\LEDD_vs_REMperc_ADV.png
Gruppo D

In [9]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Carica i dataset
df_dem = pd.read_excel(r"C:\Users\Lorenzo\Desktop\scoring/demographics/summary_extracted.xlsx")
df_sleep = pd.read_excel(r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx")

# Unisci i dataset usando "ID" e "Patient"
df = pd.merge(df_dem, df_sleep, left_on="ID", right_on="Patient")

# Creazione della variabile binaria per DYS
# Supponiamo che in 'Group' del file demografico (Group_x) il valore 'DYS' indichi la presenza di discinesie
df['DYS_flag'] = df['Group_x'].apply(lambda x: 1 if x == 'DYS' else 0)

# Per comodità, rinominiamo alcune colonne per evitare spazi e caratteri speciali
df = df.rename(columns={
    "Sleep Efficiency (%)": "SleepEff",
    "REM%": "REM_perc",
    "Disease_Duration": "DiseaseDuration"
})

# Esempio di regressione logistica univariata per LEDD
model_ledd = smf.logit("DYS_flag ~ LEDD", data=df).fit(disp=False)
print(model_ledd.summary())

# Esegui regressioni univariate per le altre variabili di interesse (Age, DiseaseDuration, Gender, SleepEff, REM_perc)
# Nota: per Gender, se è una stringa (es. "Male"/"Female"), conviene codificarla:
df['Gender_num'] = df['Gender'].apply(lambda x: 1 if str(x).lower() == 'male' else 0)

variables = ["Age", "DiseaseDuration", "LEDD", "Gender_num", "SleepEff", "REM_perc"]

selected_vars = []
for var in variables:
    formula = f"DYS_flag ~ {var}"
    model_uni = smf.logit(formula, data=df).fit(disp=False)
    p_val = model_uni.pvalues[var]
    print(f"Variabile: {var} - p-value: {p_val:.3f}")
    if p_val < 0.2:
        selected_vars.append(var)

# Se la letteratura suggerisce l'inclusione di alcune variabili, assicurati che siano presenti
# Ad esempio, se 'Age' non fosse selezionato ma è ritenuto importante:
if "Age" not in selected_vars:
    selected_vars.append("Age")

print("Variabili selezionate per il modello multivariato:", selected_vars)

# Costruisci il modello di regressione logistica multivariata
formula_multi = "DYS_flag ~ " + " + ".join(selected_vars)
model_multi = smf.logit(formula_multi, data=df).fit(disp=False)
print(model_multi.summary())

# Calcola Odds Ratios e 95% CI
params = model_multi.params
conf = model_multi.conf_int()
OR = pd.DataFrame({
    "OR": np.exp(params),
    "2.5%": np.exp(conf[0]),
    "97.5%": np.exp(conf[1])
})
print(OR)


                           Logit Regression Results                           
Dep. Variable:               DYS_flag   No. Observations:                   23
Model:                          Logit   Df Residuals:                       21
Method:                           MLE   Df Model:                            1
Date:                Sat, 08 Mar 2025   Pseudo R-squ.:                  0.4373
Time:                        12:33:20   Log-Likelihood:                -8.8611
converged:                       True   LL-Null:                       -15.746
Covariance Type:            nonrobust   LLR p-value:                 0.0002066
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.7779      1.559     -2.423      0.015      -6.834      -0.722
LEDD           0.0063      0.003      2.438      0.015       0.001       0.011
Variabile: Age - p-value: 0.260
Variabile: DiseaseDu

LinAlgError: Singular matrix